這次我們分析「帶有 emoji 的text 」的情緒，以及哪些 「emoji 本身情緒」會不同於它「所在 text 的情緒」。

作業分成 3 個部分，分別是前處理、情緒分析、情緒衝突的emoji詞頻。

**`每一部分皆有需呈現的結果`** ，繳交之前請確認每個部分的每個配分項目都有output 😉





---



# 第一部分：前處理

1. 讀取語料 `emoji_1000.csv` ，並用 `emoji module` 將每一句文本出現的 emoji 轉成英文解釋 (demojize)

2. 讀取 emoji 解釋對應表 `emoji_defintion.csv` ，並將 **前一步的英文解釋** 轉換成繁體中文。
   
   ⚠️注意：emoji 的英文解釋前後都有冒號，轉換成中文後，用**底線**將 emoji 連接在**句尾**⚠️

   ```
    "看的真的是又愛又虐😍"
   會先變成：看的真的是又愛又虐:smiling_face_with_heart-eyes: 
   再變成：看的真的是又愛又虐_心眼笑臉    
   
   "🌟天婦羅（時蔬）"
   會先變成： 天婦羅（時蔬）:glowing_star:
   再變成： 天婦羅（時蔬）_發光的星星
   ```

3. 請整理你的資料做出一個 df，依照下方範例的欄位名稱來命名，**顯示出 df[202:207]**

   ⚠️注意：不要顯示 df.head() ❌❌❌

   ⚠️注意：當一個句子有多個 emoji 時，請以半型逗號連結成string，再將這個string放入 emoji_only 欄（參考下方範例中 id 205 的句子）


> 配分方式
- step 1.  (10%)
- step 2. (10%)
- step 3. (20%)  df[202:207]

In [1]:
## emoji module
%pip install emoji --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df=pd.read_csv('emoji_1000.csv')
df

,Unnamed: 0,sentence
0,0,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️
1,1,為今日難吃的涼麵做個紀錄😤
2,2,前幾天的照片真的慘不忍睹😂
3,3,🌟天婦羅（時蔬）
4,4,看的真的是又愛又虐😍
...,...,...
994,994,今年的我決定未來都要把每一天都過得開心愉快🤩
995,995,沒有之一😍😍
996,996,很容易在沒有乾的情況下染到眼皮上💦
997,997,一直很想要的格子西外終於入手😌很保暖！


In [3]:
import emoji

In [4]:
demojized_list=[]

for sentence in df['sentence']:
    demojized = emoji.demojize(sentence)
    demojized_list.append(demojized)
demojized_list[202:207]

['我就突然啜泣哭了:loudly_crying_face:（我原本以為只會默默流眼淚',
 '每次都說粗重的我來，妳在旁邊等:face_with_tears_of_joy:',
 '不見不散:two_hearts:',
 '放個假沒想到也換我發生悲劇了:loudly_crying_face::loudly_crying_face::loudly_crying_face:',
 '希望照片妳滿意啦:oncoming_fist_light_skin_tone:']

In [5]:
#將 emoji 置於句尾

import re 

# Define a regular expression pattern to match emojis
emoji_pattern = re.compile(r':\w+:')

end_emoji=[]

for sentence in demojized_list:
    matches = emoji_pattern.findall(sentence)
    
    if matches:
        for emoji in matches:
            sentence = sentence.replace(emoji, '')
        
        sentence = sentence.strip() + ' ' + ' '.join(matches)
    
    end_emoji.append(sentence)
end_emoji[202:207]

['我就突然啜泣哭了（我原本以為只會默默流眼淚 :loudly_crying_face:',
 '每次都說粗重的我來，妳在旁邊等 :face_with_tears_of_joy:',
 '不見不散 :two_hearts:',
 '放個假沒想到也換我發生悲劇了 :loudly_crying_face: :loudly_crying_face: :loudly_crying_face:',
 '希望照片妳滿意啦 :oncoming_fist_light_skin_tone:']

In [6]:
emoji_df = pd.read_csv('emoji_definition.csv')
emoji_df

,emoji,en,clean_en,zh-TW
0,🥇,:1st_place_medal:,1st place medal,第一名獎牌
1,🥈,:2nd_place_medal:,2nd place medal,第二名獎牌
2,🥉,:3rd_place_medal:,3rd place medal,第三名獎牌
3,🅰,:A_button_(blood_type):,A button (blood type),A鈕（血型）
4,♈,:Aries:,Aries,白羊座
...,...,...,...,...
1196,💛,:yellow_heart:,yellow heart,黃心
1197,🟨,:yellow_square:,yellow square,黃色方塊
1198,🤪,:zany_face:,zany face,滑稽的臉
1199,🦓,:zebra:,zebra,斑馬


In [7]:
#轉換成 dic
new_emoji_dic = emoji_df.to_dict('index')

#怕結果太長，所以做了切片，顯示五個就好：）
short_dict = {k: v for k, v in new_emoji_dic.items() if k < 5}
print(short_dict)

{0: {'emoji': '🥇', 'en': ':1st_place_medal:', 'clean_en': '1st place medal', 'zh-TW': '第一名獎牌'}, 1: {'emoji': '🥈', 'en': ':2nd_place_medal:', 'clean_en': '2nd place medal', 'zh-TW': '第二名獎牌'}, 2: {'emoji': '🥉', 'en': ':3rd_place_medal:', 'clean_en': '3rd place medal', 'zh-TW': '第三名獎牌'}, 3: {'emoji': '🅰', 'en': ':A_button_(blood_type):', 'clean_en': 'A button (blood type)', 'zh-TW': 'A鈕（血型）'}, 4: {'emoji': '♈', 'en': ':Aries:', 'clean_en': 'Aries', 'zh-TW': '白羊座'}}


In [14]:
#text_emoji
for i, text in enumerate(end_emoji):
    for key, value in new_emoji_dic.items():
        if value['en'] in text:
            end_emoji[i] = text.replace(value['en'], f"_{value['zh-TW']}")
print(end_emoji)
end_emoji[202:207]


['還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！ _紅心', '為今日難吃的涼麵做個紀錄 _臉上有蒸汽從鼻子裡冒出來', '前幾天的照片真的慘不忍睹 _滿臉喜悅的淚水', '天婦羅（時蔬） _發光的星星', '看的真的是又愛又虐_心眼笑臉', '畢竟厚擦掉色就變薄擦了嘛 _笑得在地上打滾', '好開心我們能一起去頒獎 _笑臉與心', '時間：2021年3月24日（三）19:30-21:00 _小橙色鑽石', '奧萬大遊客中心到台電1.8公里，到溫泉約7.5公里 _暗按鈕', '所以也只能等到出社會後自己負擔矯正費用 _帶翅膀的錢', '首先是要做出最基礎的檸檬 _檸檬', '雖然最後我們清氣球清的很累 _笑得在地上打滾', '我覺得發佈會的文字光看就很有粉紅泡泡了…. _昏昏沉沉的臉 _昏昏沉沉的臉 _兩顆心', '：你男友呢？ _沒有嘴巴的臉', 'BTW高雄巨蛋某位櫃姐好沒耐心，跟她說想看磨砂膏，只問要看哪個味道，也不打算一一介紹，要不是需要真的不想跟她買，她再多推薦一下我就會敗沐浴油的說在這邊祝福大家都能遇到有耐心的櫃哥櫃姐 _滿臉喜悅的淚水 _滿臉喜悅的淚水 _紅心', 'Bling的感覺 _閃閃發光', '本來夏莉已經要回家，卻在路途中看見下雨，擔心泰武的她再次衝回醫院，意外再次被爺爺撞見，（但是我發現這邊爺爺的臉好像有稍微緩和了一點），之後的這段我覺得兩個演員都情感大炸裂，夏莉的表情跟台詞都深深安慰著泰武（好好哭） _懇求的臉', '第一次寫美食文 _雙手合十', '小年夜前夕和老公一起整理擦拭並把年花們就定位 _紅心', '結果學弟根本沒聽進去 _揚眉的臉 _揚眉的臉 _揚眉的臉', '（我姐跟同事姐姐還在那邊裝不認識然後在握手打招呼哈哈哈哈哈蟲還沒發現還在旁邊傻笑） _笑得在地上打滾', '但沒搶到票 _含淚的笑臉', '油痘肌人 _舉手的人', '例如：消水腫妝/好物分享也可以發：腫泡泡眼妝容分享 _右箭頭', '比起高中打破頭，現在真的帥多了啦好希望超級希望下次我看葉先生比賽的時候，他扣給我看喔（前提先贏球 _滿臉喜悅的淚水', '專業石頭收集員 _狗臉', '滿福水餃 _複選標記按鈕', '：啊你都會帶歷任來動物園喔？ _台灣', '封面已經太多朴志焄 _滿臉喜悅的淚水', '#極度防禦溫和防曬乳 _複選標記', '我通常早餐都會吃兩片全麥土

['我就突然啜泣哭了（我原本以為只會默默流眼淚 _大聲哭泣的臉',
 '每次都說粗重的我來，妳在旁邊等 _滿臉喜悅的淚水',
 '不見不散 _兩顆心',
 '放個假沒想到也換我發生悲劇了 _大聲哭泣的臉 _大聲哭泣的臉 _大聲哭泣的臉',
 '希望照片妳滿意啦 _迎面而來的拳頭淺膚色']

In [17]:
#emo-only
emo_only = []

for text in end_emoji:
    emoji_list = text.split('_')[1:]
    if len(emoji_list) == 1:
        emo_only.append(emoji_list[0].strip())
    else:
        emo_only.append(", ".join([emoji.strip() for emoji in emoji_list]))

emo_only[202:207]

['大聲哭泣的臉', '滿臉喜悅的淚水', '兩顆心', '大聲哭泣的臉, 大聲哭泣的臉, 大聲哭泣的臉', '迎面而來的拳頭淺膚色']

In [18]:
#text-only
text_only = []
for text in end_emoji:
    text_only.append(text.split('_')[0])      

text_only[202:207]

['我就突然啜泣哭了（我原本以為只會默默流眼淚 ',
 '每次都說粗重的我來，妳在旁邊等 ',
 '不見不散 ',
 '放個假沒想到也換我發生悲劇了 ',
 '希望照片妳滿意啦 ']

In [19]:
#sentence
df_list=df['sentence'].values.tolist()
print(df_list)

['還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️', '為今日難吃的涼麵做個紀錄😤', '前幾天的照片真的慘不忍睹😂', '🌟天婦羅（時蔬）', '看的真的是又愛又虐😍', '畢竟厚擦掉色就變薄擦了嘛🤣', '好開心我們能一起去頒獎🥰', '🔸時間：2021年3月24日（三）19:30-21:00', '🔅奧萬大遊客中心到台電1.8公里，到溫泉約7.5公里', '所以也只能等到出社會後自己負擔矯正費用💸', '首先是要做出最基礎的檸檬🍋', '雖然最後我們清氣球清的很累🤣', '我覺得發佈會的文字光看就很有粉紅泡泡了….🥴🥴💕', '😶：你男友呢？', 'BTW高雄巨蛋某位櫃姐好沒耐心，跟她說想看磨砂膏，只問要看哪個味道，也不打算一一介紹，要不是需要真的不想跟她買，她再多推薦一下我就會敗沐浴油的說😂😂在這邊祝福大家都能遇到有耐心的櫃哥櫃姐❤️', 'Bling的感覺✨', '本來夏莉已經要回家，卻在路途中看見下雨，擔心泰武的她再次衝回醫院，意外再次被爺爺撞見，（但是我發現這邊爺爺的臉好像有稍微緩和了一點），之後的這段我覺得兩個演員都情感大炸裂，夏莉的表情跟台詞都深深安慰著泰武（好好哭🥺）', '第一次寫美食文🙏', '小年夜前夕和老公一起整理擦拭並把年花們就定位❤️', '結果學弟根本沒聽進去🤨🤨🤨', '（我姐跟同事姐姐還在那邊裝不認識然後在握手打招呼哈哈哈哈哈🤣蟲還沒發現還在旁邊傻笑）', '但沒搶到票\U0001f972', '油痘肌人🙋', '例如：消水腫妝/好物分享➡️也可以發：腫泡泡眼妝容分享', '比起高中打破頭，現在真的帥多了啦😂好希望超級希望下次我看葉先生比賽的時候，他扣給我看喔（前提先贏球', '專業石頭收集員🐶', '✅滿福水餃', '🇹🇼：啊你都會帶歷任來動物園喔？', '封面已經太多朴志焄😂', '✔️#極度防禦溫和防曬乳', '我通常早餐都會吃兩片全麥土司或是多穀物（multigrain)吐司，但如果全部夾在一起菜會塞不下🤣，所以我都是直接把菜放上面夾著麵包吃，感覺像吃兩個三明治🥪🥪😍！', '▪體育', '不一樣啊完全😭😭😭😭😭', 'Ado，多麼夢幻的組合✨一起來搶鮮聽鋼琴版吧', '懶人搭法：一件洋裝輕鬆搞定。抱歉我只有半身照可以參考～～洋裝外搭配一件外套還有顯瘦的效果🤫', '🍫M&M‘s糖衣巧克力布朗尼口味', '很多都是

In [20]:
#合併

col1 = pd.Series(df_list)
col2 = pd.Series(text_only)
col3 = pd.Series(emo_only)
col4 = pd.Series(end_emoji)

emoji_analysis = pd.DataFrame({'sentence': col1, 'text_only': col2, 'emo-only': col3, 'text_emoji': col4})
emoji_analysis[202:207]

,sentence,text_only,emo-only,text_emoji
202,我就突然啜泣哭了😭（我原本以為只會默默流眼淚,我就突然啜泣哭了（我原本以為只會默默流眼淚,大聲哭泣的臉,我就突然啜泣哭了（我原本以為只會默默流眼淚 _大聲哭泣的臉
203,每次都說粗重的我來，妳在旁邊等😂,每次都說粗重的我來，妳在旁邊等,滿臉喜悅的淚水,每次都說粗重的我來，妳在旁邊等 _滿臉喜悅的淚水
204,不見不散💕,不見不散,兩顆心,不見不散 _兩顆心
205,放個假沒想到也換我發生悲劇了😭😭😭,放個假沒想到也換我發生悲劇了,"大聲哭泣的臉, 大聲哭泣的臉, 大聲哭泣的臉",放個假沒想到也換我發生悲劇了 _大聲哭泣的臉 _大聲哭泣的臉 _大聲哭泣的臉
206,希望照片妳滿意啦👊🏻,希望照片妳滿意啦,迎面而來的拳頭淺膚色,希望照片妳滿意啦 _迎面而來的拳頭淺膚色




---



# 第二部分：情緒分析

1. 跑完前處理後，我們會得到一個 df，每一列都是一句 text 的相關資料。

2. 請完成以下 setup 的 block，載入並利用此 classifier，以 「欄位」為單位，分別跑 text_only, emo_only, text_emoji 三組的情緒分析。
3. 成功跑出情緒分析結果後，請與原本的 df 合併，**顯示出一個 senti_df**，請依照下方 `回答範例 DataFrame` 來命名欄位。
   
   - text_only 的情緒分析結果 存在 text_senti 欄
   - emo_only 的情緒分析結果 存在 emoji_senti 欄	
   - text_emoji 的情緒分析結果 存在 txt_emo_senti 欄


3. 利用前一步做好的 df，分別輸出 (text_senti, emoji_senti)、(text_senti, txt_emo_senti)、(emoji_senti, txt_emo_senti) 的 contingency table，**總共3️⃣個** 。請參考下方範例 block。

> 配分方式
- DataFrame (25%)
- contingency table (15%，每一個table 5%)

In [21]:
'''''
Model source: https://huggingface.co/liam168/c2-roberta-base-finetuned-dianping-chinese/commit/952591d4ffb6df7b674eba74c4e2bb5dc9cb3128#d2h-611991
'''''

## 安裝模型
!pip install transformers datasets

## 載入模型 (請參考連結中的示範code)
from transformers import AutoModelForSequenceClassification , AutoTokenizer, pipeline
model_name = "liam168/c2-roberta-base-finetuned-dianping-chinese"
class_num = 2
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=class_num)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 載入classifier (請參考連結中的示範code)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
## 回答範例 DataFrame
senti_df.head()

NameError: name 'senti_df' is not defined

In [23]:
list_version = emoji_analysis.values.tolist()
list_version[:5]

[['還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️',
  '還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！ ',
  '紅心',
  '還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！ _紅心'],
 ['為今日難吃的涼麵做個紀錄😤',
  '為今日難吃的涼麵做個紀錄 ',
  '臉上有蒸汽從鼻子裡冒出來',
  '為今日難吃的涼麵做個紀錄 _臉上有蒸汽從鼻子裡冒出來'],
 ['前幾天的照片真的慘不忍睹😂', '前幾天的照片真的慘不忍睹 ', '滿臉喜悅的淚水', '前幾天的照片真的慘不忍睹 _滿臉喜悅的淚水'],
 ['🌟天婦羅（時蔬）', '天婦羅（時蔬） ', '發光的星星', '天婦羅（時蔬） _發光的星星'],
 ['看的真的是又愛又虐😍', '看的真的是又愛又虐', '心眼笑臉', '看的真的是又愛又虐_心眼笑臉']]

In [24]:
sentiment_list = []
for row in list_version:
    text_only = classifier(row[1])
    emo_only = classifier(row[2])
    text_emoji = classifier(row[3])
    
    row.append(text_only[0]['label'])
    row.append(emo_only[0]['label'])
    row.append(text_emoji[0]['label'])
    sentiment_list.append(row)
sentiment_list



[['還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️',
  '還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！ ',
  '紅心',
  '還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！ _紅心',
  'positive',
  'negative',
  'positive'],
 ['為今日難吃的涼麵做個紀錄😤',
  '為今日難吃的涼麵做個紀錄 ',
  '臉上有蒸汽從鼻子裡冒出來',
  '為今日難吃的涼麵做個紀錄 _臉上有蒸汽從鼻子裡冒出來',
  'negative',
  'positive',
  'negative'],
 ['前幾天的照片真的慘不忍睹😂',
  '前幾天的照片真的慘不忍睹 ',
  '滿臉喜悅的淚水',
  '前幾天的照片真的慘不忍睹 _滿臉喜悅的淚水',
  'negative',
  'negative',
  'negative'],
 ['🌟天婦羅（時蔬）',
  '天婦羅（時蔬） ',
  '發光的星星',
  '天婦羅（時蔬） _發光的星星',
  'negative',
  'positive',
  'negative'],
 ['看的真的是又愛又虐😍',
  '看的真的是又愛又虐',
  '心眼笑臉',
  '看的真的是又愛又虐_心眼笑臉',
  'negative',
  'positive',
  'negative'],
 ['畢竟厚擦掉色就變薄擦了嘛🤣',
  '畢竟厚擦掉色就變薄擦了嘛 ',
  '笑得在地上打滾',
  '畢竟厚擦掉色就變薄擦了嘛 _笑得在地上打滾',
  'negative',
  'positive',
  'positive'],
 ['好開心我們能一起去頒獎🥰',
  '好開心我們能一起去頒獎 ',
  '笑臉與心',
  '好開心我們能一起去頒獎 _笑臉與心',
  'positive',
  'positive',
  'positive'],
 ['🔸時間：2021年3月24日（三）19:30-21:00',
  '時間：2021年3月24日（三）19:30-21:00 ',
  '小橙色鑽石',
  '時間：2021年3月24日（三）19:30-21:00 _小橙色鑽石',
  'positive',
  'positive',
  'positive'],
 [

In [25]:
sentiment_df = pd.DataFrame(sentiment_list, columns=['sentence', 'text_only', 'emo_only', 'text_emoji', 'text_senti', 'emoji_senti', 'txt_emo_senti'])
sentiment_df.head()

,sentence,text_only,emo_only,text_emoji,text_senti,emoji_senti,txt_emo_senti
0,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！,紅心,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！ _紅心,positive,negative,positive
1,為今日難吃的涼麵做個紀錄😤,為今日難吃的涼麵做個紀錄,臉上有蒸汽從鼻子裡冒出來,為今日難吃的涼麵做個紀錄 _臉上有蒸汽從鼻子裡冒出來,negative,positive,negative
2,前幾天的照片真的慘不忍睹😂,前幾天的照片真的慘不忍睹,滿臉喜悅的淚水,前幾天的照片真的慘不忍睹 _滿臉喜悅的淚水,negative,negative,negative
3,🌟天婦羅（時蔬）,天婦羅（時蔬）,發光的星星,天婦羅（時蔬） _發光的星星,negative,positive,negative
4,看的真的是又愛又虐😍,看的真的是又愛又虐,心眼笑臉,看的真的是又愛又虐_心眼笑臉,negative,positive,negative


In [ ]:
## 回答範例 Contingency Table 
# 此處只顯示(text_senti, txt_emo_senti)這組的結果
# ❗️❗️❗️❗️❗️❗️總共要輸出 3 組❗️❗️❗️❗️❗️❗️

txt_emo_senti,negative,positive
text_senti,,
negative,194,64
positive,93,648


In [26]:
tb1 = pd.crosstab(sentiment_df['text_senti'], sentiment_df['emoji_senti'])
tb1

emoji_senti,negative,positive
text_senti,,
negative,111,151
positive,223,514


In [27]:
tb2 = pd.crosstab(sentiment_df['emoji_senti'], sentiment_df['txt_emo_senti'])
tb2

txt_emo_senti,negative,positive
emoji_senti,,
negative,181,153
positive,106,559


In [28]:
#(text_senti, txt_emo_senti)

tb3 = pd.crosstab(sentiment_df['text_senti'], sentiment_df['txt_emo_senti'])
tb3

txt_emo_senti,negative,positive
text_senti,,
negative,195,67
positive,92,645




---



# 第三部分：衝突 emoji 詞頻

請你找出 `emoji_senti` 和 `txt_emo_senti` 兩個欄位被標成不同情緒的 text 們，並計算出現在這些 text 中的 emoji 的頻率。

**最後用 DataFrame 列出最高頻的 10 個 emoji**



```
🔔 會用到的欄位: emo_only, emoji_senti, txt_emo_senti

🔴 注意：若同個 text 中有多個 emoji，請將這些 emoji 分次都算進去 🔴 
舉例來說， 
有一個 row 是 "推坑大家🐹🐹"
他的 [emoji_senti] 是 positive、[txt_emo_senti] 是 negative
所以這一筆 text 是符合我們要求的。

由於他的 [emo_only] 欄位是 "倉鼠,倉鼠"
計算頻率時 “倉鼠” 要被算到兩次。(而不是一次)
```

> 配分方式：20% (每一個emoji2%)

In [38]:
sentiment_df.head(13)

,sentence,text_only,emo_only,text_emoji,text_senti,emoji_senti,txt_emo_senti
0,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！❤️,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！,紅心,還你一個清爽暢通的寶鼻(⁎⁍̴̛ᴗ⁍̴̛⁎)！_紅心,positive,negative,positive
1,為今日難吃的涼麵做個紀錄😤,為今日難吃的涼麵做個紀錄,臉上有蒸汽從鼻子裡冒出來,為今日難吃的涼麵做個紀錄_臉上有蒸汽從鼻子裡冒出來,negative,positive,negative
2,前幾天的照片真的慘不忍睹😂,前幾天的照片真的慘不忍睹,滿臉喜悅的淚水,前幾天的照片真的慘不忍睹_滿臉喜悅的淚水,negative,negative,negative
3,🌟天婦羅（時蔬）,天婦羅（時蔬）,發光的星星,天婦羅（時蔬）_發光的星星,negative,positive,negative
4,看的真的是又愛又虐😍,看的真的是又愛又虐,心眼笑臉,看的真的是又愛又虐_心眼笑臉,negative,positive,negative
5,畢竟厚擦掉色就變薄擦了嘛🤣,畢竟厚擦掉色就變薄擦了嘛,笑得在地上打滾,畢竟厚擦掉色就變薄擦了嘛_笑得在地上打滾,negative,positive,positive
6,好開心我們能一起去頒獎🥰,好開心我們能一起去頒獎,笑臉與心,好開心我們能一起去頒獎_笑臉與心,positive,positive,positive
7,🔸時間：2021年3月24日（三）19:30-21:00,時間：2021年3月24日（三）19:30-21:00,小橙色鑽石,時間：2021年3月24日（三）19:30-21:00_小橙色鑽石,positive,positive,positive
8,🔅奧萬大遊客中心到台電1.8公里，到溫泉約7.5公里,奧萬大遊客中心到台電1.8公里，到溫泉約7.5公里,暗按鈕,奧萬大遊客中心到台電1.8公里，到溫泉約7.5公里_暗按鈕,positive,positive,positive
9,所以也只能等到出社會後自己負擔矯正費用💸,所以也只能等到出社會後自己負擔矯正費用,帶翅膀的錢,所以也只能等到出社會後自己負擔矯正費用_帶翅膀的錢,negative,positive,negative


# BONUS (10分）

試著提出一個可行的pipeline（不需寫出真實code 但也不要是空泛的概念）

這個 pipeline要抓出「以emoji造成諷刺效果」的句子，

也就是句子要保留emoji、必須辨識出哪些句子存在諷刺

(e.g., "我覺得他們做得很好欸🤗", "哇真厲害🙄"）

> 也可以使用 chatGPT，說不定他能提供非常有用的意見喔😇😇😇
